<a href="https://colab.research.google.com/github/MickDobbsKildavin2/MachineLearning/blob/main/Individual_Assignment_train_Run2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries.

In [1]:
import os                           # For file processing.
import csv                          # For reading csv files.
import shutil
from keras.models import Model
#from keras.optimizers import Adam
from keras.optimizers import adam_v2

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [7]:
from google.colab import drive
drive.mount ('/content/gdrive')

!unzip gdrive/My\ Drive/data/archive.zip > /dev/null

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
#Set directory values
#split_size=0.1
#print('split_size =',split_size)

# Get the current directory.
current_dir = os.getcwd()

# the data will be written to a working directory.
working_dir    = os.path.join(current_dir,'data')
train_data_dir = os.path.join(current_dir,'Train')
test_data_dir  = os.path.join(current_dir,'Test')
meta_data_dir  = os.path.join(current_dir,'Meta')

print('current_dir      = ',current_dir)
print('working_dir      = ',working_dir)
print('train_data_dir   = ',train_data_dir)
print('test_data_dir    = ',test_data_dir)
print('meta_data_dir    = ',meta_data_dir)
meta_csv_file = (os.path.join(current_dir,'Meta.csv'))
print('meta_csv_file    = ',meta_csv_file)


current_dir      =  /content
working_dir      =  /content/data
train_data_dir   =  /content/Train
test_data_dir    =  /content/Test
meta_data_dir    =  /content/Meta
meta_csv_file    =  /content/Meta.csv


### To properly reference the 43 classes in the metadata, create a directory structure to allow the data generators to match the classes to the test, validation and training datasets.

In [8]:
# Open the meta.csv file and use the ClassId to build a directory structure that will allow the 
# build of a class subset to match the training and test data.
with open(meta_csv_file,"r") as csvfile:
    r= csv.reader(csvfile,delimiter =',')
    # Loop through the rows
    for i,row in enumerate(r):
        if i == 0: 
            continue
        # Extract the class label
        label = row[1]
        #print('label',label)
        # Extract the image name
        img_name = label + '.png'
        #print('img_name',img_name)

        # Creayte the sub-folder if it does not exist
        dest = os.path.join(meta_data_dir,label)
        if not os.path.isdir(dest):
            os.makedirs(dest)
        
        # Copy files to the relevant working directory   
        to_move = os.path.join(meta_data_dir,img_name)
        #print('to_move',to_move)
        #print('dest',dest)
        shutil.copy(to_move,dest)

print('metadata config complete.')        

FileNotFoundError: ignored

In [9]:
class_subset = sorted(os.listdir(meta_data_dir))[:43] # Using only the first 10 classes
print('class_subset    = ',class_subset)

class_subset    =  ['.~lock.ClassesInformation.ods#', '.~lock.ClassesInformationStrong.ods#', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '5', '6', '7']


### Create a training data generator to resize the images to the VGG16 expected size and to split the training data into:
  -  A training set
  -  A validation set

Use an 85% to 15% split for now.

This code is based on the working example at : 
https://www.learndatasci.com/tutorials/convolutional-neural-networks-image-classification/

In [10]:
BATCH_SIZE = 64
# The default input size for this model is 224x224.
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
dim = (IMAGE_WIDTH, IMAGE_HEIGHT)

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

### Now use the data generator to create a training dataset.  This will resize the images to the VGG16 expected size and create a training dataset for the training model.  The resizing is to match the data that VGG16 was trained with.

validation_split=0.15 so the test split will be 85%


In [12]:
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

Found 30888 images belonging to 43 classes.


### Create a validation data generator to resize the images to the VGG16 expected size and to assign some of the original training data to the validation set.

validation_split=15%

In [13]:
validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

Found 5441 images belonging to 43 classes.


In a similar way to the metadata used to set up the classification set, teh training data needs to be given a directory structure the same as the training data.

In [14]:
### Create a test dataset.
test_csv_file = (os.path.join(current_dir,'Test.csv'))
print('test_csv_file    = ',test_csv_file)


test_csv_file    =  /content/Test.csv


In [15]:
# prepare_test(source_test_folder,test_csv_file)
with open(test_csv_file,"r") as csvfile:
    r= csv.reader(csvfile,delimiter =',')
    for i,row in enumerate(r):
        if i == 0: 
            continue
        label = row[-2]
        #print('label =',label)
        img_name = row[-1]
        #print('img_name =',img_name)
        # Creayte the sub-folder if it does not exist
        #print('test_data_dir =',test_data_dir)
        dest = os.path.join(test_data_dir,label)
        if not os.path.isdir(dest):
            os.makedirs(dest)
        #print('dest          =',dest)

        # Copy files to the relevant working directory   
        to_move = os.path.join(current_dir,img_name)
        #print('to_move       =',to_move)
        shutil.move(to_move,dest)
print('test data re-config complete.')        

test data re-config complete.


Create a validation data generator to resize the images to the VGG16 expected size and to assign some of the original training data to the validation set.

In [16]:
testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 11700 images belonging to 43 classes.


Now that the data is set up to match the VGG16 model, we can create a new model based on VGG16 

This is the key part of implementing the concept of Transfer Learning.

This model will use pre-trained layers to process the GTSRB data, using a model built for image recognition to retrain a model specifically for the traffic sign data.

This create_model proc is taken straight from https://www.learndatasci.com/tutorials/convolutional-neural-networks-image-classification/

In [18]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Set a learning rate and the shape for the input layer.    
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)
optim_1 = adam_v2.Adam(learning_rate=0.001)
n_classes=43

# Set the values for the validation.    
n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 15

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)
vgg_model



In [19]:
# Having problems running this on colab and locally.  The only way it works for me is to do a dynamic install.
!pip install livelossplot --quiet

from livelossplot.inputs.keras import PlotLossesCallback
plot_loss_1 = PlotLossesCallback()

Set a checkpoint setting.

In [20]:
# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

Stop training when the model has stopped improving.

We want to minimize the loss so that is what is monitored for improvement here.

In [21]:
# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [22]:
%%time
#print('%%time',%%time)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 13.8 µs


In [ ]:
%%time 

vgg_history = vgg_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            #epochs=50,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=482,
                            validation_steps=85,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)

Epoch 1/15


In [ ]:
# Reset our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
#from livelossplot import PlotLossesTensorFlowKeras
from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

ModuleNotFoundError: ignored

In [ ]:
!pip install livelossplot --quiet

In [ ]:
%matplotlib inline

from time import sleep
import numpy as np

from livelossplot import PlotLosses

#from livelossplot import PlotLossesCallback

In [ ]:
plot_loss_1 = PlotLosses()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

NameError: ignored